# File Structure

This notebook contains assumptions, EDA, reflections, and ETL code that I made during work on climate datasets.

In [1]:
import pandas as pd
import numpy as np

In [43]:
import sqlalchemy, pyodbc

import urllib

from sqlalchemy import create_engine, Column, Integer
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from urllib.parse import quote_plus

from config import DB_USERNAME, DB_PASSWORD, DB_SERVER, DB_NAME, DB_DRIVER, DB_SCHEMA

ModuleNotFoundError: No module named 'config'

In [2]:
pd.set_option('display.max_columns', None)

# Data Load, EDA and Preparation Sections

Use a copy of `Theatres.xlsx` as to not risk corrupting the original

In [24]:
df_cities = pd.read_csv('archive\GlobalLandTemperaturesByCity.csv')

In [26]:
df_cities.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [27]:
print(df_cities.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   dt                             object 
 1   AverageTemperature             float64
 2   AverageTemperatureUncertainty  float64
 3   City                           object 
 4   Country                        object 
 5   Latitude                       object 
 6   Longitude                      object 
dtypes: float64(2), object(5)
memory usage: 459.2+ MB
None


In [28]:
df_fra = df_cities[df_cities["Country"] == "France"]

In [29]:
print(df_fra.duplicated().sum())

0


In [30]:
df_fra

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
117012,1743-11-01,7.478,1.866,Aix En Provence,France,44.20N,4.47E
117013,1743-12-01,NaN,NaN,Aix En Provence,France,44.20N,4.47E
117014,1744-01-01,NaN,NaN,Aix En Provence,France,44.20N,4.47E
117015,1744-02-01,NaN,NaN,Aix En Provence,France,44.20N,4.47E
117016,1744-03-01,NaN,NaN,Aix En Provence,France,44.20N,4.47E
...,...,...,...,...,...,...,...
8045376,2013-05-01,10.477,0.406,Villeurbanne,France,45.81N,5.77E
8045377,2013-06-01,15.970,0.453,Villeurbanne,France,45.81N,5.77E
8045378,2013-07-01,20.531,0.367,Villeurbanne,France,45.81N,5.77E
8045379,2013-08-01,18.633,0.417,Villeurbanne,France,45.81N,5.77E


In [31]:
df_fra.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [32]:
df_fra['City'].value_counts()

City
Aix En Provence         3239
Amiens                  3239
Nîmes                   3239
Nancy                   3239
Nantes                  3239
Nice                    3239
Orléans                 3239
Paris                   3239
Perpignan               3239
Reims                   3239
Rennes                  3239
Rouen                   3239
Saint Étienne           3239
Strasbourg              3239
Toulon                  3239
Toulouse                3239
Tours                   3239
Mulhouse                3239
Montpellier             3239
Metz                    3239
Clermont Ferrand        3239
Angers                  3239
Besançon                3239
Bordeaux                3239
Boulogne Billancourt    3239
Brest                   3239
Caen                    3239
Dijon                   3239
Marseille               3239
Grenoble                3239
Le Havre                3239
Le Mans                 3239
Lille                   3239
Limoges                 3239
Lyon     

In [33]:
df_fra.isnull().sum()

dt                                  0
AverageTemperature               2628
AverageTemperatureUncertainty    2628
City                                0
Country                             0
Latitude                            0
Longitude                           0
dtype: int64

In [34]:
df_fra.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,113976.000000,113976.000000
mean,10.337918,1.577883
std,6.280883,1.893495
min,-9.497000,0.058000
25%,5.157000,0.330000
50%,10.279000,0.699000
75%,15.887000,2.188000
max,27.174000,13.738000


In [35]:
df_fra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116604 entries, 117012 to 8045380
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   dt                             116604 non-null  object 
 1   AverageTemperature             113976 non-null  float64
 2   AverageTemperatureUncertainty  113976 non-null  float64
 3   City                           116604 non-null  object 
 4   Country                        116604 non-null  object 
 5   Latitude                       116604 non-null  object 
 6   Longitude                      116604 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.1+ MB


In [36]:
df_fra['dt'].info()

<class 'pandas.core.series.Series'>
Index: 116604 entries, 117012 to 8045380
Series name: dt
Non-Null Count   Dtype 
--------------   ----- 
116604 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


## Conversion of the dataframes' datetime columns to the correct dtypes

In [37]:
#df_fra = df_fra.columns

df_fra_datetime_formatted = df_fra.copy()

column_name = "dt"

print(df_fra_datetime_formatted[column_name].info())

try:
    df_fra_datetime_formatted[column_name] = pd.to_datetime(df_fra_datetime_formatted[column_name], format='%Y-%m-%d', errors='coerce')
except Exception as e:
    print("An exception occurred:", type(e).__name__, "–", e)

<class 'pandas.core.series.Series'>
Index: 116604 entries, 117012 to 8045380
Series name: dt
Non-Null Count   Dtype 
--------------   ----- 
116604 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB
None


In [38]:
df_fra_datetime_formatted['dt'].info()

<class 'pandas.core.series.Series'>
Index: 116604 entries, 117012 to 8045380
Series name: dt
Non-Null Count   Dtype         
--------------   -----         
116604 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.8 MB


In [39]:
for column_name in df_fra_datetime_formatted:
    post_processing_nan_count = df_fra_datetime_formatted[column_name].isnull().sum()
    pre_processing_nan_count = df_fra[column_name].isnull().sum()
    if post_processing_nan_count != pre_processing_nan_count:
        print('Values were lost in column:', column_name, '| Difference in NaN/null counts:', post_processing_nan_count - pre_processing_nan_count)

---
Upon further inspection both of these columns contain numerous entries with integer values in the 40,000s range, therefore it is safe to move forward.

---

### Loading onto Azure Server

In [41]:
df = df_fra_datetime_formatted.copy()

In [42]:
mappings = {
        'object' : 'text',
        'int64' : 'int',
        'float64' : 'decimal',
        'datetime64[ns]' : 'datetime'
}


sql_build_query_start = f"""
    CREATE TABLE [{schema}].[{table}] (
    """

sql_built_query_end = """
)
"""

sql_built_query = sql_build_query_start

for idx, column in enumerate(df.columns):
    if (str(column) == 'Unique_Case_Reference'):
        sql_built_query = sql_built_query + str(column) + ' ' + mappings[str(df.dtypes.iloc[idx])] + ' ' "primary key"
    else:
        sql_built_query = sql_built_query + str(column) + ' ' + mappings[str(df.dtypes.iloc[idx])]

    if not(idx == len(df.columns)-1):
        sql_built_query = sql_built_query + ",\n"


sql_built_query = sql_built_query + sql_built_query_end
print(sql_built_query)


CREATE TABLE [temperatures_schema].[FranceLandTemperaturesByCity] (
dt datetime,
AverageTemperature decimal,
AverageTemperatureUncertainty decimal,
City text,
Country text,
Latitude text,
Longitude text
)



In [ ]:
server   = DB_SERVER
database = DB_NAME
username = DB_USERNAME
password = DB_PASSWORD
odbc_driver = DB_DRIVER


odbc_str = 'DRIVER='+odbc_driver+';SERVER='+server+';PORT=1433;UID='+username+';DATABASE='+ database + ';PWD='+ password

pyodbc_conn_str = (
    f"Driver={odbc_driver};"
    f"Server=tcp:{server},1433;"
    f"Database={database};"
    f"Uid={username};"
    f"Pwd={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;")

sql_alchemy_connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)

try:
    connection = pyodbc.connect(pyodbc_conn_str)
    cursor = connection.cursor()

    # Create table in Azure SQL DB with pyodbc
    cursor.execute(sql_built_query)
    cursor.commit()

    # Push Dataframe to Azure SQL DB using SQL Alchemy
    engine = create_engine(sql_alchemy_connect_str)
    theatres_df.to_sql(table_name, schema=schema_name, con=engine, if_exists='replace', index=False)

except Exception as e:
    print(e)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()